In [ ]:
# Importing necessary libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing  import image
import pandas as pd

In [ ]:
# Mounting Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load and display an image from the dataset 
img = image.load_img("/content/drive/MyDrive/ project 1/DATASET/TRAIN/happy/Screenshot 2023-04-14 221802.png")
plt.imshow(img)

In [ ]:
# Check the shape of an image using OpenCV
cv2.imread("/content/drive/MyDrive/ project 1/DATASET/TRAIN/anger/Screenshot 2023-04-13 130831.png").shape

(224, 159, 3)

In [ ]:
# Define directories for training and validation datasets
train_directory = "/content/drive/MyDrive/ project 1/DATASET/TRAIN"
val_directory = "/content/drive/MyDrive/ project 1/DATASET/validation"

In [ ]:
# Rescaling pixel values for normalization (1/255 to scale values between 0 and 1)
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

In [ ]:
# Loading training dataset and preprocessing images
train_dataset = train.flow_from_directory('/content/drive/MyDrive/ project 1/DATASET/TRAIN',
                           target_size=(224,224),
                           batch_size = 32,
                           class_mode = 'categorical')

# Loading validation dataset and preprocessing images
validation_dataset = validation.flow_from_directory('/content/drive/MyDrive/ project 1/DATASET/validation',
                           target_size=(224,224),
                           batch_size = 32,
                           class_mode = 'categorical')

In [ ]:
# Checking class indices for the dataset
train_dataset.class_indices

In [ ]:
# Checking class labels for the dataset
train_dataset.classes

In [ ]:
# Building the CNN model
from tensorflow.keras import Model
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    tf.keras.layers.Dropout(0.1),

                                    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    tf.keras.layers.Dropout(0.1),

                                    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    tf.keras.layers.Dropout(0.1),

                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(256, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(5, activation='softmax')
                                  ])

In [ ]:
# Compiling the model with categorical cross-entropy loss and RMSprop optimizer
model.compile(loss='categorical_crossentropy',
              optimizer = RMSprop(learning_rate=0.001),
              metrics =['accuracy'])
model.summary()

In [ ]:
# Displaying the model summary
history = model.fit(train_dataset,
                      epochs = 10,
                      validation_data = validation_dataset)

In [ ]:
# Saving the trained model
from tensorflow.keras.models import load_model
my_model=model.save('/content/drive/MyDrive/ project 1/saved_model/emotion_detection.h5')

In [ ]:
# Plotting training and validation loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'],label='train loss')
plt.plot(history.history['val_loss'],label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# Plotting training and validation accuracy
plt.plot(history.history['accuracy'],label='train acc')
plt.plot(history.history['val_accuracy'],label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_loss')

In [ ]:
# Predicting emotions for test images
import os
import matplotlib.pyplot as plt

dir_path = "/content/drive/MyDrive/ project 1/DATASET/TEST (1)"


def predict_emotion(val):
    if val == 0:
        print('predicted as anger')
    elif val == 1:
        print('predicted as fear')
    elif val == 2:
        print('predicted as happy')
    elif val == 3:
        print('predicted as neutral')
    else:
        print('predicted as sad')


for i in os.listdir(dir_path):
    img = image.load_img(dir_path + '//' + i, target_size=(224, 224))
    plt.imshow(img)
    plt.show()

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    pred = model.predict(images)
    val = np.argmax(pred)
    predict_emotion(val)


In [ ]:
# Evaluate the model on the validation dataset
loss, accuracy = model.evaluate(validation_dataset)

# Print the accuracy of the model
print('Accuracy on validation dataset:', accuracy)


In [ ]:
# Loading and preprocessing the test dataset for evaluation
test_directory = "/content/drive/MyDrive/ project 1/DATASET/TEST"

test = ImageDataGenerator(rescale=1/255)

test_dataset = test.flow_from_directory(test_directory,
target_size=(224,224),
batch_size = 3,
class_mode = 'categorical')

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_dataset)

print('Test accuracy:', test_acc)

In [ ]:
class_labels = ['anger','fear','happy','neutral','sad']

In [ ]:
test_directory = "/content/drive/MyDrive/ project 1/DATASET/TEST"
test_dataset = validation.flow_from_directory(test_directory,
                                              target_size=(224,224),
                                              batch_size = 1,
                                              class_mode = 'categorical',
                                              shuffle=False)
test_labels = test_dataset.classes

# Predicting class labels for the test dataset
test_predictions = model.predict(test_dataset)
test_predictions = np.argmax(test_predictions, axis=1)


# Confusion matrix visualization
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

import seaborn as sns
sns.heatmap(conf_matrix, annot=True, fmt="d", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()